## REST API Examples

In [1]:
%%writefile Makefile

# Author: harsha
# Purpose: Provide simple "make" targets for developers
# See README for details about each target.

# Default goal runs the "all" target
.DEFAULT_GOAL := all

.PHONY: all
all: clean lint run

.PHONY: setup
setup:
	@echo "Starting  setup"
	pip install -r requirements.txt
	mkdir -p resp
	@echo "Completed setup"

.PHONY: lint
lint:
	@echo "Starting  lint"
	find . -name "*.py" | xargs pylint
	find . -name "*.py" | xargs black -l 82 --check
	@echo "Completed lint"

.PHONY: run
run:
	@echo "Starting  run"
	python basic_get.py
	python basic_log.py
	python print_response.py
	python session_test.py
	python cache_control.py
	python redirect_automatic.py
	python redirect_manual.py
	python get_cisco_sdwan_devices.py
	python get_some_pokemon.py
	python get_all_pokemon.py
	@echo "Completed run"

.PHONY: clean
clean:
	@echo "Starting  clean"
	find . -name "*.pyc" | xargs rm
	rm -f resp/*
	@echo "Starting  clean"

Writing Makefile


In [2]:
%%writefile basic_get.py

#!/usr/bin/env python

"""
Author: harsha
Purpose: Trivial script to test Python requests.
"""

import sys
import requests


def main(site):
    """
    Performs a GET request on the supplied website.
    """

    # Issue HTTP GET request, check for success, and print body
    resp = requests.get(site)

    # If not "ok", raise HTTPError, otherwise do nothing
    # Code: ~/environments/pyreq/lib/python3.6/site-packages/requests/models.py
    resp.raise_for_status()
    print(resp.text)


if __name__ == "__main__":

    # Ensure there are at least 2 arguments; extract the
    # second one (first one after the script name)
    if len(sys.argv) >= 2:
        main(sys.argv[1])

    # Insufficient CLI args supplied; use author's homepage by default
    else:
        main("http://njrusmc.net")

Writing basic_get.py


In [3]:
! python basic_get.py

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml" >
<head>
    <title>njrusmc.net</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0"/>
    <style type="text/css">li {margin: 4px 0;}</style>
    <link rel="alternate" type="application/atom+xml"
      title="Atom feed for njrusmc.net" href="blog/atom.xml" />
</head>
<body>
  <h1>Nick Russo Website Homepage</h1>
  <p>
  Welcome! This website provides references to my publicly-available
  professional work. Why the simple design?
  </p>
  <ul>
    <li>
      <b>Accessible: </b>Not everyone has high-speed Internet access.
      You won't find clunky images, videos, or advertisements.
      This site is mobile-friendly.
    </li>
    <li>
      <b>Navigable: </b>Most of this website is plain text
      and can be searched using browser search tools.
      Every resource on this website is within two clicks.
    </li>
    <li>
      <b>Useful: </b>This website is not a personal sales channel.
	  It cont

In [4]:
%%writefile basic_log.py
#!/usr/bin/env python

"""
Author: harsha
Purpose: Trivial script to test Python requests with logging enabled.
"""

import logging
import sys
import requests


def main(site):
    """
    Performs a GET request on the supplied website with
    additional logging enabled.
    """

    # Create a basic logger using a common configuration
    # Example format: 2020-05-20 07:12:38 INFO Hello world!
    logging.basicConfig(
        format="%(asctime)s %(levelname)-8s %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        level=logging.DEBUG,
    )
    logger = logging.getLogger()

    # Optional debugging statement
    # breakpoint()  # py3.7+
    # import pdb; pdb.set_trace()  # py3.6-

    # Issue HTTP GET request, check for success, and print body
    logger.info("Sending GET to %s", site)
    resp = requests.get(site)
    logger.info("Completed GET to %s", site)

    # If not "ok", raise HTTPError, otherwise do nothing
    resp.raise_for_status()
    logger.info("Response headers: %s", resp.headers)


if __name__ == "__main__":

    # Ensure there are at least 2 arguments; extract the
    # second one (first one after the script name)
    if len(sys.argv) >= 2:
        main(sys.argv[1])

    # Insufficient CLI args supplied; use author's homepage by default
    else:
        main("http://njrusmc.net")

Writing basic_log.py


In [5]:
%%writefile cache_conntrol.py

#!/usr/bin/env python

"""
Author: 
Purpose: Demonstrate basic HTTP cache-control mechanisms using a variety
of URLs with different cache-control characteristics.
"""

import logging
import time
import requests
from cachecontrol import CacheControl
from print_response import print_response


def main():
    """
    Execution begins here.
    """

    # Use our standard logger template
    logging.basicConfig(
        format="%(asctime)s %(levelname)-8s %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        level=logging.DEBUG,
    )
    logger = logging.getLogger()

    # Specify list of test files to download.
    # Go to http://njrusmc.net/cache/cache.html to see all test files
    base_url = "http://njrusmc.net/cache"
    test_list = [
        "zero128k_public60.test",  # Cache-Control: public, max-age=60
        "zero128k_nostore.test",  # Cache-Control: no-store
    ]

    # For each file, run two GET requests, and use the logger to print out
    # the relevant information as requests are processed
    for test_file in test_list:

        # Assemble the complete URL to feed into the HTTP GET request
        url = f"{base_url}/{test_file}"

        # Create the cached session object, which automatically intereprets
        # caching-related headers (requests doesn't do it natively)
        cached_sess = CacheControl(requests.session())

        # Print information from first run, include key headers
        logger.info("First GET to %s", url)
        resp = cached_sess.get(url)
        resp.raise_for_status()
        print_response(resp, dump_body=False)
        print(f"\n\n{'*' * 80}\n\n")

        # Slight delay just to show the cache timer countdown
        # Print information from second run, but focus is on background debugs
        time.sleep(2)
        logger.info("Second GET to %s", url)
        resp = cached_sess.get(url)
        resp.raise_for_status()
        print_response(resp, dump_body=False)
        print(f"\n\n{'*' * 80}\n\n")


if __name__ == "__main__":
    main()

Writing cache_conntrol.py


In [6]:
%%writefile get_cisco_sdwan_devices.py
#!/usr/bin/env python

"""
Author: harsha
Purpose: Basic consumption of Cisco SD-WAN REST API using the
public Cisco DevNet sandbox.
"""

import requests
from requests.exceptions import HTTPError
from print_response import print_response


def main():
    """
    Execution begins here.
    """

    # Define base URL and disable SSL warnings (self-signed cert)
    api_path = "https://sandbox-sdwan-1.cisco.com"
    requests.packages.urllib3.disable_warnings()

    # These credentials are supplied by Cisco DevNet on the sandbox page:
    # https://developer.cisco.com/sdwan/learn/
    creds = {"j_username": "devnetuser", "j_password": "RG!_Yw919_83"}

    # Create session and issue POST request. "data" will unpack the dict
    # into key/value pairs. Also, disable SSL validation
    sess = requests.session()
    auth = sess.post(f"{api_path}/j_security_check", data=creds, verify=False)

    # Optional debugging statement
    # breakpoint()  # py3.7+
    # import pdb; pdb.set_trace()  # py3.6-

    # An authentication request has failed if we receive a failing return code
    # OR if there is any text supplied in the response. Failing authentications
    # often return code 200 (OK) but include a lot of HTML content, indicating a
    # a failure. If a failure does occur, exit the program using code 1.
    if not auth.ok or auth.text:
        raise HTTPError("Authentication failed")

    # Authentication succeeded; issue HTTP GET to collect devices
    devices = sess.get(f"{api_path}/dataservice/device", verify=False)
    devices.raise_for_status()
    print_response(devices, filename="get_cisco_sdwan_devices")


if __name__ == "__main__":
    main()

Writing get_cisco_sdwan_devices.py


In [7]:
%%writefile redirect_manual.py

#!/usr/bin/env python

"""
Author: 
Purpose: Demonstrate how requests handles HTTP redirects for
a typical GET request.
"""

import requests
from print_response import print_response


def main():
    """
    Execution begins here.
    """

    # This links to my "Python 3 By Example" course on O'Reilly
    resp1 = requests.get("http://njrusmc.net/r/sltreq", allow_redirects=False)
    resp1.raise_for_status()
    print_response(resp1, dump_body=False)

    # If >= 300, it's a redirect. Errors are impossible here; those
    # would have been caught by raise_for_status() earlier
    if resp1.status_code >= 300:

        # Extract next URL from Location header
        url = resp1.headers["Location"]

        ### Perform security checks on "url" if desired ###

        # Issue another GET request to the next URL
        resp2 = requests.get(url, allow_redirects=False)
        resp2.raise_for_status()
        print_response(resp2, dump_body=False)


if __name__ == "__main__":
    main()

Writing redirect_manual.py


In [8]:
%%writefile session_test.py

#!/usr/bin/env python

"""
Author: 
Purpose: Demonstrate how a long-lived TCP session works with requests.
"""

import logging
import requests


def main():
    """
    Execution begins here.
    """

    # Use our standard logger template
    logging.basicConfig(
        format="%(asctime)s %(levelname)-8s %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        level=logging.DEBUG,
    )
    logger = logging.getLogger()

    # Define generic URL to test
    url = "http://njrusmc.net"

    # Issue two HTTP GET requests without using a session
    logger.info("Individual, stateless requests")
    requests.get(url)
    requests.get(url)

    # Now use a session; setup occurs only once
    logger.info("Long-lived, stateful TCP session")
    sess = requests.session()
    sess.get(url)
    sess.get(url)


if __name__ == "__main__":
    main()

Writing session_test.py


In [9]:
%%writefile print_response.py

#!/usr/bin/env python

"""
Author: 
Purpose: Simple printing mechanism for HTTP responses that come back
from the 'requests' library for troubleshooting/learning.
"""

import json
import requests


def print_response(resp, filedir="resp", filename=None, dump_body=True):
    """
    Print the key details from a given response. Set dump_body to true
    to reveal the body content in the most appropriate format based on
    the response's Content-Type header.
    """

    # For context, print the original request
    # ... and a line of dashes of equal length
    header = f"\n\n{resp.request.method} {resp.request.url}"
    print(header)
    print(len(header) * "-")

    # Print the status code and reason, along with the elapsed time
    print(f"Result: {resp.status_code}/{resp.reason}")
    print(f"Elapsed time: {resp.elapsed.microseconds} us")

    # Iterate over all kv-pairs in the headers dictionary and print them
    print("HTTP headers:")
    for header_name, header_value in resp.headers.items():
        print(f"  - {header_name}: {header_value}")

    # Print the number of redirects. If any exist, print out the URL/status
    print(f"HTTP redirect count: {len(resp.history)}")
    for hist in resp.history:
        print(f"  - {hist.url} -> {hist.status_code}/{hist.reason}")

    # Optionally dump the body; useful for plain text responses (not files)
    if dump_body:
        # First, determine the content type, defaulting to "html"
        content_type = resp.headers.get("Content-Type", "html")

        # If a filename was not supplied, create a dynamic one using
        # the method name and the in-memory ID of the response object
        if not filename:
            filename = f"{resp.request.method}_{id(resp)}".lower()

        # Define the entire filepath using the directory and name
        filepath = f"{filedir}/{filename}"

        # Based on the content type, create different files.
        # Add more options if you want ...
        if "html" in content_type:
            filepath += ".html"
            with open(filepath, "w") as handle:
                handle.write(resp.text)
        elif "json" in content_type:
            filepath += ".json"
            with open(filepath, "w") as handle:
                json.dump(resp.json(), handle, indent=2)

        print(f"HTTP body written to {filepath}")


def main():
    """
    Performs a quick test on the print_response() function.
    """

    resp = requests.get("http://njrusmc.net")
    resp.raise_for_status()
    print_response(resp, filename="get_nick_website")


if __name__ == "__main__":
    main()
    

Writing print_response.py
